## Documentação

https://networkx.org/documentation/stable/reference/classes/multidigraph.html

https://pypi.org/project/networkx-query/

https://networkx.org/documentation/stable//reference/introduction.html

https://stackoverflow.com/questions/15644684/best-practices-for-querying-graphs-by-edge-and-node-attributes-in-networkx


In [51]:
import networkx as nx
import pandas as pd
from pyvis.network import Network
#import sqlalchemy
import ibm_db
import ibm_db_dbi

In [2]:
#conectando ao BigSql
# COLOCAR CREDENCIAIS
db = ibm_db.connect("DATABASE=BIGSQL;HOSTNAME=bigsql.pro.intra.rs.gov.br;PORT=32051;PROTOCOL=TCPIP;UID=XXXXX;PWD=XXXXXX;", "", "")
conn = ibm_db_dbi.Connection(db)   

In [7]:
# buscando os dados das arestas / quem se relaciona com quem
query = "select * from PROCERGS_MILENA_VILLAR.RELACIONAMENTO2;"
df = pd.read_sql_query(query, conn)
#informando quais colunas da tabela serão o source, target e demais atributos; create_using específica qual é o tipo do grafo
G = nx.from_pandas_edgelist(df, source = 'SRC', target = 'TGT', edge_attr=['REL','WEIGHT'], create_using=nx.MultiDiGraph())

In [23]:
#VISUALIZANDO AS ARESTAS E RELACIONAMENTOS
G.edges(data = True)

OutMultiEdgeDataView([(2, 1, {'REL': 'co-autores', 'WEIGHT': 4}), (2, 3, {'REL': 'pai', 'WEIGHT': 5}), (1, 2, {'REL': 'co-autores', 'WEIGHT': 4}), (1, 3, {'REL': 'mãe', 'WEIGHT': 5}), (1, 4, {'REL': 'mãe', 'WEIGHT': 5}), (1, 16, {'REL': 'irmã', 'WEIGHT': 3}), (1, 10, {'REL': 'irmão', 'WEIGHT': 3}), (3, 13, {'REL': 'co-autores', 'WEIGHT': 1}), (3, 12, {'REL': 'co-autores', 'WEIGHT': 1}), (3, 4, {'REL': 'irmã', 'WEIGHT': 3}), (4, 3, {'REL': 'irmã', 'WEIGHT': 3}), (4, 13, {'REL': 'co-autores', 'WEIGHT': 1}), (4, 12, {'REL': 'co-autores', 'WEIGHT': 1}), (4, 9, {'REL': 'co-autores', 'WEIGHT': 3}), (4, 8, {'REL': 'co-autores', 'WEIGHT': 3}), (10, 16, {'REL': 'irmão', 'WEIGHT': 3}), (10, 11, {'REL': 'co-autores', 'WEIGHT': 4}), (10, 12, {'REL': 'pai', 'WEIGHT': 5}), (10, 13, {'REL': 'pai', 'WEIGHT': 5}), (10, 1, {'REL': 'irmão', 'WEIGHT': 3}), (16, 10, {'REL': 'irmão', 'WEIGHT': 3}), (16, 1, {'REL': 'irmã', 'WEIGHT': 3}), (11, 10, {'REL': 'co-autores', 'WEIGHT': 4}), (11, 12, {'REL': 'mãe', '

In [41]:
# plotar o grafo
nt = Network("400px", "900px", notebook = True, directed = False)
nt.from_nx(G)
nt.show("g.html")

## Adicionando atributos aos nós

In [9]:
#buscando os labels dos nós(vértices)
query = "select * from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
df_node = pd.read_sql_query(query, conn)

In [8]:
df_node.head()

,ID,NOME,SZ,GP,LAT,LONGT,P_ARMA,TIPO_ARMA,TIPO_CRIME,NUM_VITIMAS
0,2,Eloi,5,1,-29.881725,-51.164042,SIM,2,NÃO,0
1,7,Inês,15,3,-29.916482,-51.146204,NÃO,0,NÃO,0
2,9,Carolina,15,3,-29.916482,-51.146204,SIM,3,HOMICIDIO,3
3,11,Elis,5,4,-29.897718,-51.170391,SIM,3,TRAF DROG,0
4,14,Maria,2,5,-29.897718,-51.170391,NÃO,0,NÃO,0


In [10]:
#transformando a tabela em um dicionário para atribuir labels aos IDs dos nós
nodelabel = df_node.set_index("ID").T.to_dict('records')[0]
# atribuindo o atributo label
nx.set_node_attributes(G, nodelabel, 'title')
G.nodes(data=True)

NodeDataView({2: {'title': 'Eloi'}, 1: {'title': 'Vivianne'}, 3: {'title': 'Jade'}, 4: {'title': 'Camila'}, 10: {'title': 'Fabio'}, 16: {'title': 'Virginia'}, 11: {'title': 'Elis'}, 12: {'title': 'Felipe'}, 13: {'title': 'Brenda'}, 15: {'title': 'Ney'}, 14: {'title': 'Maria'}, 17: {'title': 'Joaquim'}, 18: {'title': 'Marisa'}, 7: {'title': 'Inês'}, 19: {'title': 'Maiara'}, 20: {'title': 'Ligia'}, 5: {'title': 'Eduardo'}, 6: {'title': 'Ricardo'}, 8: {'title': 'Barbara'}, 9: {'title': 'Carolina'}})

In [12]:
#buscando os tamanhos dos nós(vértices)
query = "select ID,SZ from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
a = pd.read_sql_query(query, conn)

In [13]:
#transformando a tabela em um dicionário para atribuir os tamanhos aos IDs dos nós
nodeatrib = a.set_index("ID").T.to_dict('records')[0]
# atribuindo o atributo label
nx.set_node_attributes(G, nodeatrib, 'size')
G.nodes(data=True)

NodeDataView({2: {'title': 'Eloi', 'size': 5}, 1: {'title': 'Vivianne', 'size': 15}, 3: {'title': 'Jade', 'size': 5}, 4: {'title': 'Camila', 'size': 15}, 10: {'title': 'Fabio', 'size': 5}, 16: {'title': 'Virginia', 'size': 2}, 11: {'title': 'Elis', 'size': 5}, 12: {'title': 'Felipe', 'size': 1}, 13: {'title': 'Brenda', 'size': 1}, 15: {'title': 'Ney', 'size': 2}, 14: {'title': 'Maria', 'size': 2}, 17: {'title': 'Joaquim', 'size': 15}, 18: {'title': 'Marisa', 'size': 15}, 7: {'title': 'Inês', 'size': 15}, 19: {'title': 'Maiara', 'size': 15}, 20: {'title': 'Ligia', 'size': 15}, 5: {'title': 'Eduardo', 'size': 5}, 6: {'title': 'Ricardo', 'size': 5}, 8: {'title': 'Barbara', 'size': 15}, 9: {'title': 'Carolina', 'size': 15}})

In [15]:
#buscando os grupos dos nós(vértices)
query = "select ID,GP from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
g = pd.read_sql_query(query, conn)

In [16]:
#transformando a tabela em um dicionário para atribuir os grupos aos IDs dos nós
nodeatrib = g.set_index("ID").T.to_dict('records')[0]
# atribuindo o atributo label
nx.set_node_attributes(G, nodeatrib, 'group')
G.nodes(data=True)

NodeDataView({2: {'title': 'Eloi', 'size': 5, 'group': 1}, 1: {'title': 'Vivianne', 'size': 15, 'group': 1}, 3: {'title': 'Jade', 'size': 5, 'group': 1}, 4: {'title': 'Camila', 'size': 15, 'group': 2}, 10: {'title': 'Fabio', 'size': 5, 'group': 4}, 16: {'title': 'Virginia', 'size': 2, 'group': 6}, 11: {'title': 'Elis', 'size': 5, 'group': 4}, 12: {'title': 'Felipe', 'size': 1, 'group': 4}, 13: {'title': 'Brenda', 'size': 1, 'group': 4}, 15: {'title': 'Ney', 'size': 2, 'group': 5}, 14: {'title': 'Maria', 'size': 2, 'group': 5}, 17: {'title': 'Joaquim', 'size': 15, 'group': 7}, 18: {'title': 'Marisa', 'size': 15, 'group': 7}, 7: {'title': 'Inês', 'size': 15, 'group': 3}, 19: {'title': 'Maiara', 'size': 15, 'group': 8}, 20: {'title': 'Ligia', 'size': 15, 'group': 8}, 5: {'title': 'Eduardo', 'size': 5, 'group': 2}, 6: {'title': 'Ricardo', 'size': 5, 'group': 3}, 8: {'title': 'Barbara', 'size': 15, 'group': 3}, 9: {'title': 'Carolina', 'size': 15, 'group': 3}})

In [18]:
#buscando o atributo coordenadas dos nós(vértices)
query = "select * from PROCERGS_MILENA_VILLAR.COORDENADAS;"
at_node = pd.read_sql_query(query, conn)
at_node.head()

,ID,LAT,LONGT
0,7,-29.916482,-51.146204
1,8,-29.916482,-51.146204
2,9,-29.916482,-51.146204
3,10,-29.897718,-51.170391
4,11,-29.897718,-51.170391


In [19]:
#transformando a tabela em um dicionário
c = at_node.set_index("ID").T.to_dict('list')
#adicionando o atributo coordenada
nx.set_node_attributes(G, c, 'coord')
G.nodes(data=True)

NodeDataView({2: {'title': 'Eloi', 'size': 5, 'group': 1, 'coord': [-29.881724532142304, -51.164042431648795]}, 1: {'title': 'Vivianne', 'size': 15, 'group': 1, 'coord': [-29.881724532142304, -51.164042431648795]}, 3: {'title': 'Jade', 'size': 5, 'group': 1, 'coord': [-29.881724532142304, -51.164042431648795]}, 4: {'title': 'Camila', 'size': 15, 'group': 2, 'coord': [-29.894813954047603, -51.15983506048432]}, 10: {'title': 'Fabio', 'size': 5, 'group': 4, 'coord': [-29.897717760996297, -51.17039117397637]}, 16: {'title': 'Virginia', 'size': 2, 'group': 6, 'coord': [-30.029295930685592, -51.201077060480124]}, 11: {'title': 'Elis', 'size': 5, 'group': 4, 'coord': [-29.897717760996297, -51.17039117397637]}, 12: {'title': 'Felipe', 'size': 1, 'group': 4, 'coord': [-29.897717760996297, -51.17039117397637]}, 13: {'title': 'Brenda', 'size': 1, 'group': 4, 'coord': [-29.897717760996297, -51.17039117397637]}, 15: {'title': 'Ney', 'size': 2, 'group': 5, 'coord': [-29.897717760996297, -51.17039117

In [21]:
#buscando TIPO DE CRIME, NUM. VÍTIMAS dos nós(vértices)
query = "select ID,P_ARMA,TIPO_ARMA,TIPO_CRIME,NUM_VITIMAS from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
g = pd.read_sql_query(query, conn)

In [22]:
#transformando a tabela em um dicionário para atribuir aos IDs dos nós
nodeatrib = g.set_index("ID").T.to_dict()
# atribuindo o atributo 
nx.set_node_attributes(G, nodeatrib)
G.nodes(data=True)

NodeDataView({2: {'title': 'Eloi', 'size': 5, 'group': 1, 'coord': [-29.881724532142304, -51.164042431648795], 'P_ARMA': 'SIM', 'TIPO_ARMA': '2', 'TIPO_CRIME': 'NÃO', 'NUM_VITIMAS': '0'}, 1: {'title': 'Vivianne', 'size': 15, 'group': 1, 'coord': [-29.881724532142304, -51.164042431648795], 'P_ARMA': 'SIM', 'TIPO_ARMA': '2', 'TIPO_CRIME': 'ASSALTO', 'NUM_VITIMAS': '5'}, 3: {'title': 'Jade', 'size': 5, 'group': 1, 'coord': [-29.881724532142304, -51.164042431648795], 'P_ARMA': 'NÃO', 'TIPO_ARMA': '0', 'TIPO_CRIME': 'NÃO', 'NUM_VITIMAS': '0'}, 4: {'title': 'Camila', 'size': 15, 'group': 2, 'coord': [-29.894813954047603, -51.15983506048432], 'P_ARMA': 'SIM', 'TIPO_ARMA': '4', 'TIPO_CRIME': 'SEQUESTRO', 'NUM_VITIMAS': '3'}, 10: {'title': 'Fabio', 'size': 5, 'group': 4, 'coord': [-29.897717760996297, -51.17039117397637], 'P_ARMA': 'SIM', 'TIPO_ARMA': '4', 'TIPO_CRIME': 'HOMICIDIO', 'NUM_VITIMAS': '10'}, 16: {'title': 'Virginia', 'size': 2, 'group': 6, 'coord': [-30.029295930685592, -51.2010770

In [52]:
# plotar o grafo
nt = Network("400px", "900px", notebook = True)
nt.toggle_hide_edges_on_drag(False) # esconde as arestas quando clica com o mouse
#nt.barnes_hut() #tipo de layout
# nt.show_buttons(filter_=['physics'])
nt.from_nx(G)
nt.show("ga.html")

## Consultas

In [24]:
# Descobrindo quais nós possuem relacionamento = advogado
relacionamento = [(u,v) for (u,v,d) in G.edges(data=True) if d["REL"] == 'advog']
list(relacionamento)

[(15, 2),
 (15, 3),
 (15, 5),
 (14, 1),
 (14, 10),
 (17, 19),
 (17, 20),
 (17, 4),
 (18, 7),
 (18, 6)]

In [26]:
# identificando os nós
print(G.nodes[14]['title'])

Maria


In [27]:
# buscando nós que possuem o relacionamento 'co-autores' e estão no grupo '1'
q = set(n for u,v,d in G.edges(data=True)
               if d['REL']=='co-autores'
               for n in (u, v)
               if G.nodes[n]['group'] == 1)
print(q)

{1, 2, 3}


In [29]:
#Consultando atributo dos nós
#método 1
grupo = (n for n in G if G.nodes[n]['NUM_VITIMAS'] > '2')
print(list(grupo))

[1, 4, 16, 13, 17, 5, 9]


In [30]:
#Consultando atributo dos nós
# método 2
grupo = (n for n,d in G.nodes(data=True) if d['group'] == 2 and d['NUM_VITIMAS'] > '2')
print(list(grupo))

[4, 5]


In [32]:
# métodos para encontrar relações que saem de um nó específico e vão para outro (vizinhos)
# 1º metodo
G[3]

AdjacencyView({13: {0: {'REL': 'co-autores', 'WEIGHT': 1}}, 12: {0: {'REL': 'co-autores', 'WEIGHT': 1}}, 4: {0: {'REL': 'irmã', 'WEIGHT': 3}}})

In [33]:
# 2º metodo
G.adj[3]

AdjacencyView({13: {0: {'REL': 'co-autores', 'WEIGHT': 1}}, 12: {0: {'REL': 'co-autores', 'WEIGHT': 1}}, 4: {0: {'REL': 'irmã', 'WEIGHT': 3}}})

In [34]:
# 3º metodo
list(G.neighbors(3))

[13, 12, 4]

In [35]:
# 4º metodo
list(G.successors(3))

[13, 12, 4]

In [36]:
# método para encontrar relações que chegam em um nó específico 
list(G.predecessors(3))

[1, 2, 4, 13, 12, 15]

In [55]:
# vizinhos (sucessores e predecessores) do nó 3, raio = 1
nt = Network("400px", "900px", notebook = True)
H= nx.generators.ego_graph(G, 3, radius=1, center = True, undirected = True)
nt.from_nx(H)
nt.show("h.html")

In [37]:
#Número de vítimas dos vizinhos (predecessores) de 3
for n in G.predecessors(3):
    print(G.nodes[n]['NUM_VITIMAS'])

5
0
3
4
1
0


In [38]:
#Número de vítimas dos vizinhos (sucessores)  de 3
for n in G.successors(3):
    print(G.nodes[n]['NUM_VITIMAS'])

4
1
3


In [39]:
# Vizinhos (sucessores) de 3 que possuem mais de 3 vítimas
[y for y in [x for x in G.successors(3)] 
 if 'NUM_VITIMAS' in G.nodes[y] and G.nodes[y]['NUM_VITIMAS'] >= '3']

[13, 4]

In [40]:
# Vizinhos (predecessores) de 3 que possuem mais de 3 vítimas
[y for y in [x for x in G.predecessors(3)]
 if 'NUM_VITIMAS' in G.nodes[y] and G.nodes[y]['NUM_VITIMAS'] >= '3']

[1, 4, 13]

In [45]:
# Quem são as pessoas que têm armas?
arma = (n for n in G if G.nodes[n]['P_ARMA'] == 'SIM')
print(sorted(arma))


[1, 2, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20]


In [ ]:
# vizinhos comuns entre dois nós
nx.common_neighbors(G,1,4)
# não implementado para grafos direcionados